In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import tensorflow_probability as tfp 
import seaborn as sns 
import geopandas 
import datetime
import scipy as sp

2024-09-22 20:32:56.242274: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'geopandas'

In [ ]:
sns.set_theme()

In [ ]:
plt.rcParams["figure.figsize"] = (15, 10)

In [ ]:
hpai_main = pd.read_csv('data_files/combinedDataset20221130 _1_.csv')
hpai_wild = pd.read_csv('data_files/wildBirdData20221130 _1_.csv')
hpai_poultry = pd.read_excel('data_files/GBPR Poultry Premises Extract 01Dec22 _1_.xlsx')

## Preview the data

In [ ]:
hpai_main.head(5)

In [ ]:
hpai_main.info()

Comments: 
- unique ID for each farm
- dates (resolution is daily) 
    - format is DD/MM/YYYY
    - reported date for initial infection? 
    - culling dates for end of infectious period (can be longer than 1 day - should factor into the model, decay term? ) 
    - clarify: infDate, confDate, reportDate
- herd details (types of birds present - cat var in model to tell difference in susceptibility) 
- herd size
- spatial component (have geolocation and regional area - can impute from geolocation if need be)

In [ ]:
hpai_poultry.head()

In [ ]:
hpai_poultry.info()

Comments: 

- spatial data for farms 
- long/lat coordinates for mapping

In [ ]:
hpai_wild.head()

In [ ]:
hpai_wild.info()

In [ ]:
hpai_wild[['Date Received']].dropna()

Comments: 

- dead wild bird reports w/ accompanying HPAI test result 
- long/lat location of carcass 
- species of wild bird (are some species more likely to be the cause?)

# EDA and Data Cleaning

## Time filtering

In [ ]:
# Time variables for hpai_main:
# reportDate
# infDate
# confDate
# cullStart
# cullEnd

hpai_main_dates = hpai_main[['reportDate', 'infDate',
                                                                                      'confDate', 'cullStart', 'cullEnd']].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'), axis=1)
hpai_main_dates[['reportDate', 'infDate', 'confDate', 'cullStart', 'cullEnd']].tail()

In [ ]:
hpai_main_dates[['reportDate', 'infDate', 'confDate', 'cullStart', 'cullEnd']] = hpai_main_dates[['reportDate', 'infDate', 'confDate', 'cullStart', 'cullEnd']].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))

In [ ]:
sns.histplot(data = hpai_main_dates, x = 'reportDate')

In [ ]:
# most cases are in 2020 onwards so we filter out the 2017 wave 
hpai_main_dates = hpai_main_dates[hpai_main_dates.reportDate >= datetime.datetime(2020,1, 1, 0,0,0)]

In [ ]:
sns.histplot(data = hpai_main_dates, x = 'reportDate', bins = 36)
# Change the axis labels and title
plt.xlabel('Month')
plt.ylabel('Number of cases')
plt.title('HPAI case counts')
plt.savefig('hpai_case_counts.png')

In [ ]:
# Time variables for hpai_wild:
# Date Received
# Test date
# Date PHE informed

hpai_wild[['Date Received', 'Test date', 'Date PHE informed']] = hpai_wild[['Date Received', 'Test date', 'Date PHE informed']].apply(
    lambda x: pd.to_datetime(x, infer_datetime_format=True), axis=1)

## Geopandas attempts

In [ ]:
sns.scatterplot(data=hpai_main, x = 'Long', y = 'Lat', style = 'region', hue='premisesType')

In [ ]:
sns.scatterplot(data=hpai_wild, x = 'easting', y = 'northing', hue='Final Result')

In [ ]:
geo_hpai_main = geopandas.GeoDataFrame(
    hpai_main, geometry=geopandas.points_from_xy(hpai_main.Long, hpai_main.Lat))

In [ ]:
geo_hpai_wild = geopandas.GeoDataFrame(
    hpai_wild, geometry=geopandas.points_from_xy(hpai_wild.longitude, hpai_wild.latitude))

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))


# restrict to UK.
ax = world[world.name == 'United Kingdom'].plot(
    color='white', edgecolor='black')

# plot our ``GeoDataFrame``.
geo_hpai_wild[geo_hpai_wild['Final Result'] == 'Positive'].plot(
    ax=ax, color='red', alpha=0.5, label='Wild bird Infected',  marker='x')
geo_hpai_wild[geo_hpai_wild['Final Result'] != 'Positive'].plot(
    ax=ax, color='green', alpha=0.3, label='Wild bird *Not* Infected',  marker='.')
geo_hpai_main.plot(ax=ax, color='blue', marker='s',
                   label='Poultry Farms', alpha=0.5)

plt.title('Wild bird deaths in relation to poultry farms in the UK')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(loc="upper left")
plt.savefig('wild_bird_occurences.png')

# Extracting modeling data 

Proposed model: SEINR 

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

# restrict to UK.
ax = world[world.name == 'United Kingdom'].plot(
    color='white', edgecolor='black')

# plot our ``GeoDataFrame``.
geo_hpai_wild[geo_hpai_wild['Final Result'] == 'Positive'].plot(
    ax=ax, color='red', alpha=0.5, label='Wild bird Infected',  marker='x')
geo_hpai_wild[geo_hpai_wild['Final Result'] != 'Positive'].plot(
    ax=ax, color='green', alpha=0.3, label='Wild bird *Not* Infected',  marker='.')
geo_hpai_main.plot(ax=ax, color='blue', marker='s',
                   label='Poultry Farms', alpha=0.5)

plt.title('Wild bird deaths in relation to poultry farms in the UK')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(loc="upper left")
plt.savefig('wild_bird_occurences.png')

plt.

In [ ]:
uk = geopandas.read_file('Countries_December_2022_GB_BFE_1266578283381653958.zip')
ax = uk.plot(
    color='white', edgecolor='black')


# plot our ``GeoDataFrame``.
geo_hpai_wild[geo_hpai_wild['Final Result'] == 'Positive'].plot(
    ax=ax, color='red', alpha=0.5, label='Wild bird Infected',  marker='x')
geo_hpai_wild[geo_hpai_wild['Final Result'] != 'Positive'].plot(
    ax=ax, color='green', alpha=0.3, label='Wild bird *Not* Infected',  marker='.')
geo_hpai_main.plot(ax=ax, color='blue', marker='s',
                   label='Poultry Farms', alpha=0.5)

plt.title('Wild bird deaths in relation to poultry farms in the UK')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(loc="upper left")
plt.savefig('wild_bird_occurences.png')


In [ ]:
uk

In [ ]:
# spatial data - farm level
hpai_poultry[['Location Id', 'Easting', 'Northing']].to_csv('data_files/farm_locations.csv')

In [ ]:
# time data - case level (will not have every farm infected)
hpai_main_dates.to_csv('data_files/hpai_temporal_data.csv')

In [ ]:
# farm characteristics - from the hpai_main dataset so they align with the hpai_main_dates
# characteristics available for every infected/case but not for every farm (i.e. occults)! Harder to simulate
hpai_main[['approxNumBirds', 'Season', 'speciesPresent', 'businessType', 'premisesType']]

# any derivative data sets from the main file can be matched by index, o/w need to match by location? 

In [ ]:
hpai_main.info()

# Simulate data

In [ ]:
import sys 

sys.path.append('/Users/alinmorariu/Documents/Github/ContinuousTimeIndividualLevelEpidemicModels')

In [ ]:
import ContinousTimeStateTransitionModel as epi_sim

## Fake data 

In [ ]:
# location data
num_farms = 1000
np.random.seed(1356315)
location_data = np.random.uniform(low = 0.0, high = 1000.0, size = (num_farms,2))
location_data = pd.DataFrame(location_data, columns= ('Long', 'Lat'))
location_data.head()

In [ ]:
sns.scatterplot(data=location_data, x='Long', y='Lat')

In [ ]:
# farm type data 
farm_types = ['chicken', 'other']
np.random.seed(1356315)


In [ ]:
# Create hazard fn 
DTYPE = tf.float32
def pairwise_distance(farm_locations_data):
    """Compute pairwise distance matrix between farms

    Args:
        farm_locations_data (DataFrame): Lat-Long coordinates of farms

    Returns:
        tensor: tensor of Eucli dean distances between entities
                Dim = len(farm_location_data)
    """
    return tf.convert_to_tensor(
        sp.spatial.distance.squareform(
            sp.spatial.distance.pdist(farm_locations_data)
        ),
        dtype=DTYPE
    )

In [ ]:
# eg. spatial distance
pairwise_distance(farm_locations_data= location_data)

In [ ]:
# Datetimes for events 
np.random.seed(1356315)

infectious_period_range = np.arange(0, 20)
base_date = np.datetime64('2023-01-07')
infectious_dates = base_date + np.random.choice(infectious_period_range, num_farms)

infectious_dates

In [ ]:
removal_dates = infectious_dates + 4

In [ ]:
key_dates = pd.DataFrame({'inf_date': pd.to_numeric(infectious_dates), 
                          'removal_date': pd.to_numeric(removal_dates)})
key_dates

In [ ]:
key_dates.iloc[:,0]

## Summary 

In [ ]:
location_data.head()

In [ ]:
key_dates.head()

# Likelihood evaluation 

In [ ]:
def generate_waifw(infection_times, removal_times):
    """
    Compute a WAIFW (who acquired infection from who) matrix 
    given tensors of infection and removal times. 

    Args:
        infection_times (datetime): array of infection times 
        removal_times (datetime): array of removal times 

    Returns:
        tensor: len(infection_times) x len(infection_times) tensor of 1s and 0s
    """

    infection_times = tf.convert_to_tensor(infection_times)
    removal_times = tf.convert_to_tensor(removal_times)

    # use the expand_dim to do the [I,:] trick
    waifw = tf.math.logical_and(
        # compare infections to infections: I_i <= I_j
        infection_times[tf.newaxis, :] < infection_times[:, tf.newaxis],
        # compare infections to removals: I_j <= R_i
        infection_times[:, tf.newaxis] < removal_times[tf.newaxis, :]
    )
    return tf.cast(waifw,
                   dtype=DTYPE)

In [ ]:
generate_waifw(infection_times= key_dates.iloc[:,0],
              removal_times= key_dates.iloc[:,1])

In [ ]:
def generate_exposure(infection_times, removal_times):
    """
    Compute exposure matrix given two tensors of infections and removal times

    Args:
        infection_times (datetime): array of infection times 
        removal_times (datetime): array of removal times 

    Returns:
        tensor: len(infection_times) x len(infection_times) tensor of exposure
        durations 
    """

    return (
        tf.math.minimum(infection_times[:, tf.newaxis],
                        removal_times[tf.newaxis, :]) -
        tf.math.minimum(infection_times[:, tf.newaxis],
                        infection_times[tf.newaxis, :])
    )

In [ ]:
generate_exposure(infection_times= key_dates.iloc[:,0],
              removal_times= key_dates.iloc[:,1])

In [ ]:
def generate_spatial_kernel(farm_distance_matrix):
    """Compute the square exponential spatial kernel given a pairwise distance matrix

    Args:
        farm_distance_matrix (tensor): matrix of pairwise distance

    Returns:
        function: function taking in spatial pressure parameter based on 
    """
    farm_distance_matrix = tf.convert_to_tensor(farm_distance_matrix, DTYPE)
    def square_exponential_kernel(parameters):
        partial_step = tf.math.multiply(farm_distance_matrix, 1/parameters)
        return tf.math.exp(tf.math.negative(tf.math.square(partial_step)))
    return square_exponential_kernel

In [ ]:
farm_distances = pairwise_distance(farm_locations_data= location_data)
spatial_kernel = generate_spatial_kernel(farm_distance_matrix= farm_distances)

spatial_kernel(500.0)

In [ ]:
# farm type data 
np.random.seed(1356315)
farm_types = np.random.choice((0,1), size = num_farms) # chicken, o/w 

In [ ]:
def generate_regression_pressure(farm_characteristics_data = None): 
    """_summary_

    Args:
        farm_characteristics_data (_type_, optional): Factor variables for each farm unit. Defaults to None.

    Returns:
        tensor: exp(alpha + beta^T * data)
    """
    farm_characteristics_data = tf.convert_to_tensor(farm_characteristics_data, DTYPE)
    def compute_regression(parameters):
        # regression does not include the intercept term 
        regression = tf.math.multiply(farm_characteristics_data, parameters)
        expontiated_regression = tf.math.exp(regression)
        return expontiated_regression
        
    return compute_regression

In [ ]:
regression_pressure = generate_regression_pressure(farm_characteristics_data=farm_types)
regression_pressure(parameters=0.7)

In [ ]:
def generate_pairwise_hazard_fn(farm_distance_matrix, farm_characteristics_data=None):
    """_summary_

    Args:
        farm_characteristics_data (_type_): features of farms, including a 1s column for regression
        farm_locations_data (_type_): Northing-Easting coordinates of farms

    Returns:
        fn: fn which outputs a tensor of pairwise hazard rates 
    """
    spatial_kernel = generate_spatial_kernel(farm_distance_matrix)
    regression_kernel = generate_regression_pressure(farm_characteristics_data)
    
    def compute_hazard(parameters):
        # Fill in later

        # spatial component - already exponentiated!
        spatial = spatial_kernel(parameters[0])
        
        # regression component - already exponentiated!
        regression = regression_kernel(parameters[1])
        return tf.math.multiply(spatial,
                                regression
                               )

    return compute_hazard

In [ ]:
hazard_fn = generate_pairwise_hazard_fn(farm_distance_matrix=farm_distances, farm_characteristics_data=farm_types)
tf.einsum('ij -> i', hazard_fn([500.0,0.7]))

# Simulate epidemic (event)

In [ ]:
# make rate fn for simulation 
def make_rate_fn(parameters):
    # keep data variables and parameters on the outside
    def rate_fn(t, state):
        # t and state change with each iteration of the Gillespie so they are
        # a loop variable and thus inside the rate_fn
        si_rate = tf.einsum('ij -> i', hazard_fn(parameters))
        ir_rate = tf.broadcast_to([70.0], shape=si_rate.shape)
        return tf.stack([si_rate, ir_rate], axis=0)  # [R x M]
    return rate_fn

In [ ]:
make_rate_fn([500.0, 0.07])(1,np.array([[9],
                          [1],
                          [0]], dtype=np.float32) )

In [ ]:
# Define variables to govern epidemic
sir_graph = np.array([[-1, 0],
                      [1, -1],
                      [0, 1]], dtype=np.float32)


initial_state = np.array([[900],
                          [100],
                          [0]], dtype=np.float32)

In [ ]:
example_epidemic = epi_sim.ContinuousTimeStateTransitionModelSimulation(
    incidence_matrix=sir_graph,
    initial_state=initial_state,
    transition_rate_fn=make_rate_fn([500,0.07])
)

In [ ]:
# simulate the epidemic
epidemic1 = example_epidemic.simulate_continuous_time_state_transition_model()
time_stamps, transition_types, individual = example_epidemic.simulate_continuous_time_state_transition_model()

In [ ]:
time_stamps, transition_types, individual

In [ ]:
long_form_epi1 = epi_sim._compute_state(initial_state= epi_sim._expand_state(initial_state),
                      event_list= epidemic1,
                      incidence_matrix=sir_graph)

In [ ]:
long_form_epi1

In [ ]:
summary = tf.concat([tf.expand_dims(long_form_epi1[0], axis=1, name='time'),
                         tf.reduce_sum(long_form_epi1[1], axis=-1)],
                        axis=-1,
                        name="full_epidemic")
format_summary = pd.DataFrame(summary.numpy())

new_names = dict(zip(format_summary.columns,
                         ['time'] + ['S', 'I', 'R'])
                     )

format_summary = format_summary.rename(columns=new_names)

In [ ]:
format_summary.iloc[:, ].plot(x='time')
plt.title('Simulated Epidemic')

# Inference 
## Centred MCMC 

In [ ]:
@tf.function(jit_compile=True)
def centred_MCMC(target_log_prob_fn, initial_parameter_values, iter=25):
    '''
    MCMC sampler for centred parameterization of a 
    partially observed epidemic. Data generated from 
    SIR model. 
    '''

    dtype = np.float32
    initial_parameter_values = tf.convert_to_tensor(initial_parameter_values,
                                                    dtype=dtype)

    # generate initial values for missing data
    infection_times = removal_times - 5.0

    def single_iteration(parameters, infection_times):
        '''
        Define a single iteration of the sampler. We first update the parameters,
        followed by updating an infection time. 
        '''

        # 1(a) Propose new parameter values from Q(.)
        new_param = tfd.Normal(loc=parameters,
                               scale=[0.02, 0.01, 0.001, 0.01]).sample()

        # 1(b) Accept/reject
        a1 = target_log_prob_fn(new_param, infection_times) - \
            target_log_prob_fn(parameters, infection_times)

        # 1(c) Check
        def parameter_true_fn():
            return new_param, 1

        def parameter_false_fn():
            return parameters, 0

        is_param_accept = tf.math.log(tfd.Uniform().sample()) < a1

        parameters, result_param = tf.cond(is_param_accept,
                                           parameter_true_fn,
                                           parameter_false_fn)

        # 2(a) Propose new infection time from Q_i(.)
        # Part i - choose a person
        individual_ID = tfd.Categorical(
            logits=tf.zeros_like(removal_times)).sample()
        # Part ii - propose new infection time
        proposal = tfd.Exponential(rate=1.5)
        new_infectious_period = proposal.sample()

        current_infectious_periods = removal_times - infection_times
        new_infectious_periods = tf.tensor_scatter_nd_update(
            tensor=current_infectious_periods,
            indices=[[individual_ID]],
            updates=[new_infectious_period]
        )
        new_infection_times = removal_times - new_infectious_periods

        # 2(b) Accept/reject
        a2 = (
            target_log_prob_fn(parameters, new_infection_times)
            - target_log_prob_fn(parameters, infection_times)
            + proposal.log_prob(
                tf.gather(current_infectious_periods, individual_ID)
            )
            - proposal.log_prob(new_infectious_period)
        )

        # 2(c) Check
        def infection_true_fn():
            return new_infection_times, 1

        def infection_false_fn():
            return infection_times, 0

        is_infection_accept = tf.math.log(tfd.Uniform().sample()) < a2

        infection_times, result_inf = tf.cond(is_infection_accept,
                                              infection_true_fn,
                                              infection_false_fn)

        return x

    # Construct while loop here
    parameter_samples = tf.TensorArray(
        initial_parameter_values.dtype, size=iter)
    infection_times_samples = tf.TensorArray(infection_times.dtype, size=iter)
    results = tf.TensorArray(infection_times.dtype, size=iter)

    def cond(i,
             current_parameters,
             current_infection_times,
             parameter_accum,
             infection_times_accum,
             results_accum):
        return i < iter

    def body(i,
             current_parameters,
             current_infection_times,
             parameter_accum,
             infection_times_accum,
             results_accum):

        (next_parameters,
         next_infection_times,
         results) = single_iteration(parameters=current_parameters,
                                     infection_times=current_infection_times)

        parameter_accum = parameter_accum.write(i, next_parameters)
        infection_times_accum = infection_times_accum.write(
            i, next_infection_times)
        results_accum = results_accum.write(i, results)

        return (i+1,
                next_parameters,
                next_infection_times,
                parameter_accum,
                infection_times_accum,
                results_accum)

    (
        _1,
        _2,
        _3,
        parameter_samples,
        infection_times_samples,
        results) = tf.while_loop(cond=cond,
                                 body=body,
                                 loop_vars=(0,
                                            initial_parameter_values,
                                            infection_times,
                                            parameter_samples,
                                            infection_times_samples,
                                            results)
                                 )
    return parameter_samples.stack(), infection_times_samples.stack(), results.stack()

## Data transformation

In [ ]:
tf.scatter_nd(indices=tf.stack([individual, transition_types], axis=-1),
              updates=time_stamps, shape=[num_farms, 2])

In [ ]:
tf.stack([individual, transition_types], axis=-1)

In [ ]:
formatted_data = pd.DataFrame({'time_stamp':time_stamps, 'event_type':transition_types, 'ind_id':individual})

In [ ]:
formatted_data.sort_values('ind_id').reset_index 

In [ ]:
event_time_data = formatted_data.melt(id_vars=['ind_id', 'event_type'],
                                      value_vars=['time_stamp'],
                                      value_name='time_stamp').sort_values('ind_id').pivot(index='ind_id',
                                                                                           columns='event_type',
                                                                                           values='time_stamp').fillna(0.0)

In [ ]:
event_time_data
####### 

In [ ]:
np.column_stack((time_stamps, individual,transition_types))

In [ ]:
tf.gather(individual, np.argwhere(transition_types == 1).ravel())

In [ ]:
simulated_removal_times = tf.gather(
    time_stamps, np.argwhere(transition_types == 1).ravel())

In [ ]:
simulated_infection_times = tf.pad(tf.gather(time_stamps, np.argwhere(transition_types == 0).ravel()),
                                   paddings=tf.constant([[0, 100]]))

# Likelihood computation

In [ ]:
generate_waifw(infection_times=simulated_infection_times,
               removal_times=simulated_removal_times)

In [ ]:
generate_exposure(infection_times=simulated_infection_times,
                  removal_times=simulated_removal_times)

In [ ]:
simualted_exposure_mat = tf.matmul(a=generate_waifw(infection_times=simulated_infection_times, removal_times=simulated_removal_times),
          b=generate_exposure(infection_times=simulated_infection_times, removal_times=simulated_removal_times))

In [ ]:
def drop_initial_infections(infection_times):
    return np.argwhere(infection_times != 0.0).ravel()

In [ ]:
drop_initial_infections(infection_times=simulated_infection_times)

In [ ]:
tf.einsum('ij -> i', tf.gather(simualted_exposure_mat,
          indices=drop_initial_infections(infecti=n_times=simulated_infection_times)))

In [ ]:
def log_ll(spatial_data, unit_data, event_times):
    inf_times = event_times.get('infection_times')
    removal_times = event_times.get('removal_times')

    exposure_calcs = tf.matmul(a=generate_waifw(infection_times=inf_times, removal_times=removal_times),
                               b=generate_exposure(infection_times=inf_times, removal_times=removal_times))
    
    overall_event_rate = tf.einsum('ij -> ', tf.gather(exposure_calcs,
          indices=drop_initial_infections(infection_times=inf_times)))
    
    return overall_event_rate
    

In [ ]:
log_ll()